<div class="alert alert-block alert-info">
<p style="font-size:24px;text-align:center"><b>Compare the propagation methods: n0n2_srw, n0n2_lct, abcd_lct</b>
</div>

In [ ]:
import numpy as np
from scipy import interpolate

In [ ]:
set = 'old' 
# Options: 'old', 'new', 'function'
set_pump_type = 'left'
# Options: 'dual' 'left' 'right'

num_laser_slices = 10
num_crystal_slices = 10

gain = 0

# Adjust this to reflect choice of pump_type
z       = np.array([ 0.0,  0.5,  1.0, 1.5, 2.0, 2.5])/100.0
n2_plot = np.array([30.5, 18.3, 10.4, 5.9, 3.3, 1.9])
plot_fit = interpolate.splrep(z, n2_plot)

### Imports, Files, Crystal/Laser Params

In [ ]:
import sys, copy
from pykern.pkcollections import PKDict
import os

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import crystal
import rslaser.utils.srwl_uti_data as srwutil
import uti_plot

import srwlib

import scipy.constants as const
from scipy import special

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['pcolor.shading'] ='auto'
%matplotlib inline

In [ ]:
if set == 'old':
    package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data/20220218')
    ccd_name = 'photon_count_pump_off.txt'
    wfs_name = 'phase_pump_off.txt'
elif set == 'new':
    package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data/20220519')
    ccd_name = 'cut_photon_count_seed_laser.txt'
    wfs_name = 'phase_pump_off_zeroed.txt'    

if set != 'function':
    files = PKDict(
        meta = os.path.join(package_data_dir, 'meta_data.dat'),
        ccd = os.path.join(package_data_dir, ccd_name),
        wfs = os.path.join(package_data_dir, wfs_name)
    )

In [ ]:
w0 = 1.64e-3

params = PKDict(
    photon_e_ev = 1.5498, # Photon energy [eV], calculated from 800nm wavelength
    nslice      = num_laser_slices,
    pulseE      = 1.0e-6,
    tau_fwhm    = 300.0e-12 / np.sqrt(2.),
    tau_0       = 300.0e-12 / np.sqrt(2.),
    sigx_waist  = w0,
    sigy_waist  = w0,
)

L_cryst = 0.025
new_z = (L_cryst /num_crystal_slices) *(np.arange(num_crystal_slices)+0.5)

crystal_params = PKDict(
    length      = L_cryst, # [m]
    nslice      = num_crystal_slices,
    n0          = [1.76 for _ in range(num_crystal_slices)],
    n2          = interpolate.splev(new_z, plot_fit).tolist(),
    l_scale     = 0.001,
    pop_inversion_n_cells=64,
    pop_inversion_mesh_extent=0.01,  # [m]
    pop_inversion_crystal_alpha=120.0,  # [1/m], 1.2 1/cm
    pop_inversion_pump_waist=w0,  # [m]
    pop_inversion_pump_wavelength=532.0e-9,  # [m]
    pop_inversion_pump_energy=0.0211,  # [J], pump laser energy onto the crystal
    pop_inversion_pump_type=set_pump_type,
    pop_inversion_pump_gaussian_order=2.0,
    pop_inversion_pump_offset_x=0.0,
    pop_inversion_pump_offset_y=0.0,
)

cell_dx = (2.0 *crystal_params.pop_inversion_mesh_extent)/crystal_params.pop_inversion_n_cells

***
## Initial Intensity and Phase

In [ ]:
# Initialize laser pulse
if set == 'function':
    thisPulse = pulse.LaserPulse(params)
else:
    thisPulse = pulse.LaserPulse(params, files)

# Plot wavefront
lp_wfr_x = np.linspace(thisPulse.slice[0].wfr.mesh.xStart,thisPulse.slice[0].wfr.mesh.xFin,thisPulse.slice[0].wfr.mesh.nx)
lp_wfr_y = np.linspace(thisPulse.slice[0].wfr.mesh.yStart,thisPulse.slice[0].wfr.mesh.yFin,thisPulse.slice[0].wfr.mesh.ny)

In [ ]:
e_total_init = thisPulse.extract_total_2d_elec_fields()
intens_2d_init = 0.5 *const.c *const.epsilon_0 *(e_total_init.re**2.0 + e_total_init.im**2.0)

intens0 = intens_2d_init.flatten()
wfr0 = thisPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(lp_wfr_x*(1e3), lp_wfr_y*(1e3), intens_2d_init, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (initial)')

In [ ]:
phase_2d_init = thisPulse.extract_total_2d_phase()

phase0 = phase_2d_init.flatten()
wfr0 = thisPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(lp_wfr_x*(1e3), lp_wfr_y*(1e3), phase_2d_init, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (initial)')

In [ ]:
rayleigh_length = np.pi * (thisPulse.sigx_waist)**2.0 / (thisPulse._lambda0)
print('Rayleigh Length:', round(rayleigh_length,3), ' m')
print('RMS bunch length:', round(thisPulse.sig_s,3), ' m')

***
## n0n2_srw

In [ ]:
# Initialize laser pulse
if set == 'function':
    thisPulse = pulse.LaserPulse(params)
else:
    thisPulse = pulse.LaserPulse(params, files)

e_crystal = crystal.Crystal(crystal_params)

thisPulse_n0n2srw = e_crystal.propagate(thisPulse, 'n0n2_srw', gain, radial_n2=0)

x_n0n2srw = np.linspace(thisPulse_n0n2srw.slice[0].wfr.mesh.xStart,thisPulse_n0n2srw.slice[0].wfr.mesh.xFin,thisPulse_n0n2srw.slice[0].wfr.mesh.nx)
y_n0n2srw = np.linspace(thisPulse_n0n2srw.slice[0].wfr.mesh.yStart,thisPulse_n0n2srw.slice[0].wfr.mesh.yFin,thisPulse_n0n2srw.slice[0].wfr.mesh.ny)

In [ ]:
e_total_n0n2srw = thisPulse_n0n2srw.extract_total_2d_elec_fields()
intens_2d_n0n2srw = 0.5 *const.c *const.epsilon_0 *(e_total_n0n2srw.re**2.0 + e_total_n0n2srw.im**2.0)

intens0 = intens_2d_n0n2srw.flatten()
wfr0 = thisPulse_n0n2srw.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (n0n2_srw)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_n0n2srw*(1e3), y_n0n2srw*(1e3), intens_2d_n0n2srw, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (n0n2_srw)')

In [ ]:
phase_2d_n0n2srw = thisPulse_n0n2srw.extract_total_2d_phase()

phase0 = phase_2d_n0n2srw.flatten()
wfr0 = thisPulse_n0n2srw.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase (n0n2_srw)'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_n0n2srw*(1e3), y_n0n2srw*(1e3), phase_2d_n0n2srw, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (n0n2_srw)')

***
## n0n2_lct

In [ ]:
# Initialize laser pulse
if set == 'function':
    thisPulse = pulse.LaserPulse(params)
else:
    thisPulse = pulse.LaserPulse(params, files)

e_crystal = crystal.Crystal(crystal_params)

thisPulse_n0n2lct = e_crystal.propagate(thisPulse, 'n0n2_lct', gain, radial_n2=0)

x_n0n2lct = np.linspace(thisPulse_n0n2lct.slice[0].wfr.mesh.xStart,thisPulse_n0n2lct.slice[0].wfr.mesh.xFin,thisPulse_n0n2lct.slice[0].wfr.mesh.nx)
y_n0n2lct = np.linspace(thisPulse_n0n2lct.slice[0].wfr.mesh.yStart,thisPulse_n0n2lct.slice[0].wfr.mesh.yFin,thisPulse_n0n2lct.slice[0].wfr.mesh.ny)

In [ ]:
e_total_n0n2lct = thisPulse_n0n2lct.extract_total_2d_elec_fields()
intens_2d_n0n2lct = 0.5 *const.c *const.epsilon_0 *(e_total_n0n2lct.re**2.0 + e_total_n0n2lct.im**2.0)

intens0 = intens_2d_n0n2lct.flatten()
wfr0 = thisPulse_n0n2lct.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (n0n2_lct)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_n0n2lct*(1e3), y_n0n2lct*(1e3), intens_2d_n0n2lct, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (n0n2_lct)')

In [ ]:
phase_2d_n0n2lct = thisPulse_n0n2lct.extract_total_2d_phase()

phase0 = phase_2d_n0n2lct.flatten()
wfr0 = thisPulse_n0n2lct.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase (n0n2_lct)'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_n0n2lct*(1e3), y_n0n2lct*(1e3), phase_2d_n0n2lct, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (n0n2_lct)')

***
## abcd_lct

In [ ]:
n0_array = crystal_params.n0
n2_array = crystal_params.n2

crystal_params_cp = copy.deepcopy(crystal_params)
crystal_params_cp.nslice=1
crystal_params_cp.n0 = [crystal_params.n0[0]]
crystal_params_cp.n2 = [crystal_params.n2[0]]

# Calculate the ABCD matrix for the entire crystal
gamma_vals = np.zeros(num_crystal_slices)
abcd_mats = np.zeros((num_crystal_slices, 2, 2))
abcd_mat_tot_full = np.array( [[1, 0], [0, 1]] )
for j in range (num_crystal_slices):
    gamma_vals[j] = np.sqrt(n2_array[j] / n0_array[j])
    gamma_z = gamma_vals[j] * (L_cryst /num_crystal_slices)   
    abcd_mats[j, 0, 0] = np.cos(gamma_z)
    abcd_mats[j, 0, 1] = (1 / n0_array[j] / gamma_vals[j]) * np.sin(gamma_z)
    abcd_mats[j, 1, 0] = (- (n0_array[j] * gamma_vals[j])) * np.sin(gamma_z)
    abcd_mats[j, 1, 1] = np.cos(gamma_z)

for j in range (num_crystal_slices):
    abcd_mat_tot_full = np.matmul(abcd_mat_tot_full, abcd_mats[num_crystal_slices - j - 1, :, :])

crystal_params_cp.A = abcd_mat_tot_full[0][0]
crystal_params_cp.B = abcd_mat_tot_full[0][1]
crystal_params_cp.C = abcd_mat_tot_full[1][0]
crystal_params_cp.D = abcd_mat_tot_full[1][1]

In [ ]:
# Initialize laser pulse
if set == 'function':
    thisPulse = pulse.LaserPulse(params)
else:
    thisPulse = pulse.LaserPulse(params, files)

e_crystal = crystal.Crystal(crystal_params_cp)

thisPulse_abcdlct = e_crystal.propagate(thisPulse, 'abcd_lct', gain, radial_n2=0)

x_abcdlct = np.linspace(thisPulse_abcdlct.slice[0].wfr.mesh.xStart,thisPulse_abcdlct.slice[0].wfr.mesh.xFin,thisPulse_abcdlct.slice[0].wfr.mesh.nx)
y_abcdlct = np.linspace(thisPulse_abcdlct.slice[0].wfr.mesh.yStart,thisPulse_abcdlct.slice[0].wfr.mesh.yFin,thisPulse_abcdlct.slice[0].wfr.mesh.ny)

In [ ]:
e_total_abcdlct = thisPulse_abcdlct.extract_total_2d_elec_fields()
intens_2d_abcdlct = 0.5 *const.c *const.epsilon_0 *(e_total_abcdlct.re**2.0 + e_total_abcdlct.im**2.0)

intens0 = intens_2d_abcdlct.flatten()
wfr0 = thisPulse_abcdlct.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (abcd_lct)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_abcdlct*(1e3), y_abcdlct*(1e3), intens_2d_abcdlct, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (abcd_lct)')

In [ ]:
phase_2d_abcdlct = thisPulse_abcdlct.extract_total_2d_phase()

phase0 = phase_2d_abcdlct.flatten()
wfr0 = thisPulse_abcdlct.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase (abcd_lct)'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_abcdlct*(1e3), y_abcdlct*(1e3), phase_2d_abcdlct, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (abcd_lct)')